In [ ]:
import glob
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import pandas as pd

CLEANED_DIR = r"C:\Users\maian\OneDrive - The University of the West Indies, St. Augustine\Desktop\New folder\raw_files\cleaned_parquet" 

all_files = glob.glob(os.path.join(CLEANED_DIR, "*_merged.parquet"))
df = pd.concat([pd.read_parquet(f) for f in all_files], ignore_index=True)

df["sentiment"] = df["rating"].apply(lambda x: 1 if x > 3 else 0)
X = df["text"]
y = df["sentiment"]

vectorizer = TfidfVectorizer(min_df=5, max_df=0.8)
X_vec = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
